In [59]:
!pip install -U langchain-community langchain-openai faiss-cpu pypdf python-docx docx2txt openai

In [58]:
import os
from getpass import getpass
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from openai import OpenAI

In [60]:
os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")


client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


embedding = OpenAIEmbeddings()

Enter your OpenAI API key: ··········


In [61]:
# 📂 Paths
FOLDER_PATH = "/content/wedlii_docs"
INDEX_PATH = "/content/faiss_index"

# 🔑 Embeddings
embedding = OpenAIEmbeddings()

# ✅ Load existing FAISS index if available
if os.path.exists(f"{INDEX_PATH}/index.faiss"):
    print("✅ Loading existing FAISS index...")
    vectorstore = FAISS.load_local(INDEX_PATH, embedding, allow_dangerous_deserialization=True)

else:
    # 📂 If no docs folder, create it and stop
    if not os.path.exists(FOLDER_PATH):
        os.makedirs(FOLDER_PATH)
        print(f"📂 Created '{FOLDER_PATH}'. Please add Wedlii PDF/DOCX files and rerun.")
        raise SystemExit

    all_documents = []

    # 📥 Load all files inside folder
for file in os.listdir(FOLDER_PATH):
    path = os.path.join(FOLDER_PATH, file)
    file_name, ext = os.path.splitext(file)

    loader = None
    if ext.lower() == ".pdf":
        loader = PyPDFLoader(path)
    elif ext.lower() == ".docx":
        loader = Docx2txtLoader(path)
    else:
        continue  # skip unsupported formats

    try:
        documents = loader.load()

        # 🧹 Clean filename → make a nice title
        clean_title = file_name.replace("_", " ").replace("-", " ").title()

        # Attach useful metadata
        for i, doc in enumerate(documents, 1):
            doc.metadata.update({
                "source_file": file,
                "chunk_number": i,
                "page_number": i,
                "title": clean_title if clean_title else "Untitled Document"
            })

        print(f"✅ Loaded {file} → Title: {clean_title} with {len(documents)} chunks")
        all_documents.extend(documents)

    except Exception as e:
        print(f"⚠️ Could not load {file}, skipping... ({e})")
        continue

✅ Loading existing FAISS index...
✅ Loaded Vendors.pdf → Title: Vendors with 5 chunks
✅ Loaded AI Wedding Visions.pdf → Title: Ai Wedding Visions with 1 chunks
✅ Loaded Venues.pdf → Title: Venues with 6 chunks
✅ Loaded Bridal Hairstyle Types.pdf → Title: Bridal Hairstyle Types with 2 chunks
✅ Loaded BLOGS.pdf → Title: Blogs with 11 chunks
✅ Loaded Deamy_Byron_Bay_Boho_Wedding_Styled_Shoot.pdf → Title: Deamy Byron Bay Boho Wedding Styled Shoot with 1 chunks
✅ Loaded About Wedlii.pdf → Title: About Wedlii with 3 chunks
✅ Loaded Wedding Cake Types.pdf → Title: Wedding Cake Types with 2 chunks
✅ Loaded Wedlii_Colour_Palette.pdf → Title: Wedlii Colour Palette with 4 chunks
✅ Loaded Wedding Flower Types.pdf → Title: Wedding Flower Types with 2 chunks
✅ Loaded Bridal Makeup Types.pdf → Title: Bridal Makeup Types with 1 chunks


In [63]:
# 🔍 Load FAISS index
vectorstore = FAISS.load_local(INDEX_PATH, embedding, allow_dangerous_deserialization=True)

# 📊 Get the total number of chunks in FAISS
total_chunks = len(vectorstore.index_to_docstore_id)
print(f"📦 Total chunks stored in Wedlii vectorstore: {total_chunks}")

📦 Total chunks stored in Wedlii vectorstore: 63


In [64]:
# ✂️ Split documents into smaller overlapping chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = splitter.split_documents(all_documents)

# 🔍 Create FAISS index
vectorstore = FAISS.from_documents(chunks, embedding)
vectorstore.save_local(INDEX_PATH)
print("✅ FAISS index created and saved with chunked Wedlii docs!")

# 🔎 Create retriever (retrieve top 8 chunks for better context)
retriever = vectorstore.as_retriever(search_kwargs={"k": 8})

# 🤵 Wedlii Assistant Prompt
policy_prompt = """
You are a Wedlii Knowledge Assistant.
Only answer questions using the information provided in Wedlii documents.
If the answer is not found, say: "I cannot find that information in the Wedlii docs."
Be clear, concise, and professional.
"""

✅ FAISS index created and saved with chunked Wedlii docs!


In [ ]:
print("🤵 Wedlii Knowledge Chatbot ready! Type 'exit' to stop.\n")

while True:
    query = input("You: ")
    if query.lower() in ["exit", "quit"]:
        print("👋 Goodbye! Thanks for chatting with Wedlii.")
        break

    # 🔹 Classify query into one Wedlii category
    classification_prompt = f"""
    You are a smart classifier. Classify the following query into ONE of these Wedlii categories:

    1. About Wedlii
    2. AI Wedding Visions
    3. BLOGS
    4. Bridal Hairstyle Types
    5. Bridal Makeup Types
    6. Deamy_Byron_Bay_Boho_Wedding_Styled_Shoot
    7. Vendors
    8. Venues
    9. Wedding Cake Types
    10. Wedding Flower Types
    11. Wedlii_Colour_Palette

    Query: "{query}"

    Respond ONLY with the exact category name.
    """

    classify_response = client.chat.completions.create(
        model="gpt-4.1-nano",
        messages=[{"role": "system", "content": classification_prompt}],
        max_tokens=10,
        temperature=0
    )

    category = classify_response.choices[0].message.content.strip()
    print(f"🔍 GPT says this query is related to: {category}")

    # 🔹 Retrieve relevant docs (no metadata filter since 'title' wasn't saved)
    relevant_docs = vectorstore.similarity_search(query, k=1)

    # 🔹 Build context from retrieved docs
    context_list = []
    for i, doc in enumerate(relevant_docs, 1):
        context_list.append(
            f"Chunk {i} Content:\n{doc.page_content}\nSource: {doc.metadata.get('source_file')}\n"
        )
    context = "\n".join(context_list)

    print("\n=== Retrieved Chunks ===")
    for doc in relevant_docs:
        print("Content:", doc.page_content[:80], "...")
        print("Metadata:", doc.metadata)
        print("-" * 30)

    # 🔹 Build conversation
    messages = [
        {"role": "system", "content": policy_prompt},
        {
            "role": "user",
            "content": f"Category: {category}\n\nContext:\n{context}\n\nQuestion: {query}"
        }
    ]

    # 🔹 Get final answer from ChatGPT
    response = client.chat.completions.create(
        model="gpt-4.1-nano",   # you can switch to "gpt-4o-mini" for better answers
        messages=messages,
        max_tokens=300,
        temperature=0.5
    )

    print("Bot:", response.choices[0].message.content)
    print("=" * 80)


🤵 Wedlii Knowledge Chatbot ready! Type 'exit' to stop.

You: what is Naked / Semi-Naked Cakes 
🔍 GPT says this query is related to: Wedding Cake Types

=== Retrieved Chunks ===
Content: Fondant  Cakes  
●  Smooth,  polished,  ideal  for  detailed  designs.  
 
Naked ...
Metadata: {'producer': 'Skia/PDF m141 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Untitled document', 'source': '/content/wedlii_docs/Wedding Cake Types.pdf', 'total_pages': 2, 'page': 1, 'page_label': '2', 'source_file': 'Wedding Cake Types.pdf', 'chunk_number': 2, 'page_number': 2}
------------------------------
Bot: Naked / Semi-Naked Cakes are cakes with minimal frosting, with the cake layers exposed.
